In [ ]:
import pandas as pd
import numpy as np
import math
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/Hierarchy_clustered_df2.csv")
df.head()

,Year,Access to electricity (% of population),Agricultural land (% of land area),"Annual freshwater withdrawals, total (% of internal resources)",Arable land (% of land area),Forest area (% of land area),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Population growth (annual %),GDP per capita (current US$),CO2 emissions (metric tons per capita),Cluster
0,2000,4.446891,57.945817,43.015907,11.779587,1.852782,1586.591120,985.730004,74.989094,44.99,1.443803,182.174037,0.055167,1
1,2001,9.294527,57.947350,43.015907,11.779587,1.852782,1587.375364,1011.679617,72.811460,45.60,0.742517,182.174037,0.055293,1
2,2002,14.133616,57.939684,43.015907,11.771921,1.852782,1649.718098,1034.410867,79.063971,37.83,6.449321,182.174037,0.066810,1
3,2003,18.971165,58.083805,43.015907,11.916042,1.852782,1738.666619,1010.524231,70.249729,36.66,7.541019,199.643228,0.073005,1
4,2004,23.814182,58.151266,43.015907,11.983503,1.852782,1841.168267,1121.869767,70.890841,44.24,3.933178,221.830531,0.054867,1


In [ ]:
df['Cluster'].value_counts()

Cluster
1    4733
0     853
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['Cluster'], axis=1)
y = df['Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (4468, 13)
X_test shape: (1118, 13)
y_train shape: (4468,)
y_test shape: (1118,)


#KNN

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Initialize KNN classifier
knn = KNeighborsClassifier()


# Fit the classifier on the training data
knn.fit(X_train, y_train)

# Predict on the test data
y_pred = knn.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
scores = {label: {'specificity': None, 'sensitivity': None, 'precision': None, 'recall': None, 'f1_score': None, 'accuracy': None} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    scores[label]['specificity'] = specificity
    scores[label]['sensitivity'] = sensitivity
    scores[label]['precision'] = precision
    scores[label]['recall'] = recall
    scores[label]['f1_score'] = f1_score
    scores[label]['accuracy'] = accuracy

# Print evaluation metrics for each label
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", scores[label]['specificity'])
    print("Sensitivity:", scores[label]['sensitivity'])
    print("Precision:", scores[label]['precision'])
    print("Recall:", scores[label]['recall'])
    print("F1 Score:", scores[label]['f1_score'])
    print("Accuracy:", scores[label]['accuracy'])

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '1' Scores:
Specificity: 0.9935897435897436
Sensitivity: 0.998960498960499
Precision: 0.998960498960499
Recall: 0.998960498960499
F1 Score: 0.998960498960499
Accuracy: 0.998211091234347

Label '0' Scores:
Specificity: 0.998960498960499
Sensitivity: 0.9935897435897436
Precision: 0.9935897435897436
Recall: 0.9935897435897436
F1 Score: 0.9935897435897436
Accuracy: 0.998211091234347

Overall accuracy: 0.998211091234347


In [ ]:
# Initialize KNN classifier
knn = KNeighborsClassifier()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    knn.fit(X_train, y_train)

    # Predict on the test data
    y_pred = knn.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"\nFold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))



Fold 1:

Label '1' Scores:
Specificity: 0.9935897435897436
Sensitivity: 0.998960498960499
Precision: 0.998960498960499
Recall: 0.998960498960499
F1 Score: 0.998960498960499
Accuracy: 0.998211091234347

Label '0' Scores:
Specificity: 0.998960498960499
Sensitivity: 0.9935897435897436
Precision: 0.9935897435897436
Recall: 0.9935897435897436
F1 Score: 0.9935897435897436
Accuracy: 0.998211091234347

Fold 2:

Label '1' Scores:
Specificity: 0.994475138121547
Sensitivity: 0.9989316239316239
Precision: 0.9989316239316239
Recall: 0.9989316239316239
F1 Score: 0.9989316239316239
Accuracy: 0.9982094897045658

Label '0' Scores:
Specificity: 0.9989316239316239
Sensitivity: 0.994475138121547
Precision: 0.994475138121547
Recall: 0.994475138121547
F1 Score: 0.994475138121547
Accuracy: 0.9982094897045658

Fold 3:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9989406779661016
Precision: 1.0
Recall: 0.9989406779661016
F1 Score: 0.9994700582935877
Accuracy: 0.999104744852283

Label '0' Scores:
Specifi

#DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree classifier
dt = DecisionTreeClassifier()

# Fit the classifier on the training data
dt.fit(X_train, y_train)

# Predict on the test data
y_pred = dt.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': None, 'sensitivity': None, 'precision': None, 'recall': None, 'f1_score': None, 'accuracy': None} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '1' Scores:
Specificity: 0.9885714285714285
Sensitivity: 1.0
Precision: 0.9978813559322034
Recall: 1.0
F1 Score: 0.9989395546129375
Accuracy: 0.9982094897045658

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9885714285714285
Precision: 1.0
Recall: 0.9885714285714285
F1 Score: 0.9942528735632185
Accuracy: 0.9982094897045658

Overall accuracy: 0.9982094897045658


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree classifier
dt = DecisionTreeClassifier()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    dt.fit(X_train, y_train)

    # Predict on the test data
    y_pred = dt.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '1' Scores:
Specificity: 0.9871794871794872
Sensitivity: 0.998960498960499
Precision: 0.9979231568016614
Recall: 0.998960498960499
F1 Score: 0.9984415584415584
Accuracy: 0.9973166368515206

Label '0' Scores:
Specificity: 0.998960498960499
Sensitivity: 0.9871794871794872
Precision: 0.9935483870967742
Recall: 0.9871794871794872
F1 Score: 0.990353697749196
Accuracy: 0.9973166368515206
Fold 2:

Label '1' Scores:
Specificity: 0.9834254143646409
Sensitivity: 0.9989316239316239
Precision: 0.9968017057569296
Recall: 0.9989316239316239
F1 Score: 0.9978655282817502
Accuracy: 0.9964189794091316

Label '0' Scores:
Specificity: 0.9989316239316239
Sensitivity: 0.9834254143646409
Precision: 0.994413407821229
Recall: 0.9834254143646409
F1 Score: 0.9888888888888889
Accuracy: 0.9964189794091316
Fold 3:

Label '1' Scores:
Specificity: 0.9942196531791907
Sensitivity: 1.0
Precision: 0.9989417989417989
Recall: 1.0
F1 Score: 0.9994706193753309
Accuracy: 0.999104744852283

Label '0' Scores:
Spe

#NN

In [ ]:
from sklearn.neural_network import MLPClassifier
# Initialize Neural Network classifier
nn = MLPClassifier(random_state=42)

# Fit the classifier on the training data
nn.fit(X_train, y_train)

# Predict on the test data
y_pred = nn.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    scores[label]['specificity'] = specificity
    scores[label]['sensitivity'] = sensitivity
    scores[label]['precision'] = precision
    scores[label]['recall'] = recall
    scores[label]['f1_score'] = f1_score
    scores[label]['accuracy'] = accuracy

# Print evaluation metrics for each label
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", scores[label]['specificity'])
    print("Sensitivity:", scores[label]['sensitivity'])
    print("Precision:", scores[label]['precision'])
    print("Recall:", scores[label]['recall'])
    print("F1 Score:", scores[label]['f1_score'])
    print("Accuracy:", scores[label]['accuracy'])

# Summarize results
print("\nOverall accuracy:", report['accuracy'])



Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9968152866242038
Precision: 1.0
Recall: 0.9968152866242038
F1 Score: 0.9984051036682616
Accuracy: 0.9973142345568488

Label '0' Scores:
Specificity: 0.9968152866242038
Sensitivity: 1.0
Precision: 0.9831460674157303
Recall: 1.0
F1 Score: 0.9915014164305949
Accuracy: 0.9973142345568488

Overall accuracy: 0.9973142345568488


In [ ]:
# Initialize Neural Network classifier
nn = MLPClassifier(random_state=42, max_iter=1000)

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    nn.fit(X_train, y_train)

    # Predict on the test data
    y_pred = nn.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))

Fold 1:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9927234927234927
Precision: 1.0
Recall: 0.9927234927234927
F1 Score: 0.9963484611371936
Accuracy: 0.9937388193202147

Label '0' Scores:
Specificity: 0.9927234927234927
Sensitivity: 1.0
Precision: 0.9570552147239264
Recall: 1.0
F1 Score: 0.9780564263322884
Accuracy: 0.9937388193202147
Fold 2:

Label '1' Scores:
Specificity: 0.9447513812154696
Sensitivity: 1.0
Precision: 0.9894291754756871
Recall: 1.0
F1 Score: 0.9946865037194473
Accuracy: 0.991047448522829

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9447513812154696
Precision: 1.0
Recall: 0.9447513812154696
F1 Score: 0.9715909090909092
Accuracy: 0.991047448522829
Fold 3:

Label '1' Scores:
Specificity: 0.9942196531791907
Sensitivity: 0.9978813559322034
Precision: 0.9989395546129375
Recall: 0.9978813559322034
F1 Score: 0.9984101748807631
Accuracy: 0.9973142345568488

Label '0' Scores:
Specificity: 0.9978813559322034
Sensitivity: 0.9942196531791907
Precision: 0.988505747126

#Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Initialize Naive Bayes classifier (GaussianNB for continuous features)
nb = GaussianNB()

# Fit the classifier on the entire training data
nb.fit(X_train, y_train)

# Predict on the test data
y_pred = nb.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=nb.classes_)

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Extract evaluation metrics for each label
for idx, label in enumerate(nb.classes_):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '0' Scores:
Specificity: 0.8535031847133758
Sensitivity: 1.0
Precision: 0.5591054313099042
Recall: 1.0
F1 Score: 0.7172131147540984
Accuracy: 0.8764547896150403

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.8535031847133758
Precision: 1.0
Recall: 0.8535031847133758
F1 Score: 0.9209621993127148
Accuracy: 0.8764547896150403

Overall accuracy: 0.8764547896150403


In [ ]:
# Initialize Naive Bayes classifier
nb = GaussianNB()

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    nb.fit(X_train, y_train)

    # Predict on the test data
    y_pred = nb.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.8295218295218295
Precision: 1.0
Recall: 0.8295218295218295
F1 Score: 0.9068181818181817
Accuracy: 0.853309481216458

Label '0' Scores:
Specificity: 0.8295218295218295
Sensitivity: 1.0
Precision: 0.4875
Recall: 1.0
F1 Score: 0.6554621848739496
Accuracy: 0.853309481216458
Fold 2:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.8450854700854701
Precision: 1.0
Recall: 0.8450854700854701
F1 Score: 0.9160393746381007
Accuracy: 0.8701880035810206

Label '0' Scores:
Specificity: 0.8450854700854701
Sensitivity: 1.0
Precision: 0.5552147239263804
Recall: 1.0
F1 Score: 0.7140039447731755
Accuracy: 0.8701880035810206
Fold 3:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.8336864406779662
Precision: 1.0
Recall: 0.8336864406779662
F1 Score: 0.9093009820912767
Accuracy: 0.8594449418084154

Label '0' Scores:
Specificity: 0.8336864406779662
Sensitivity: 1.0
Precision: 0.5242424242424243
Recall: 1.0
F1 Score: 0.6878727634194831
Accuracy: 0

#SVM Linear

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Initialize SVM linear classifier
svm = SVC(kernel='linear')


# Fit the classifier on the training data
svm.fit(X_train, y_train)

# Predict on the test data
y_pred = svm.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Generate confusion matrix
labels = y.unique()
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Print evaluation metrics
# Iterate over indices and labels simultaneously
for idx, label in enumerate(labels):
    print(f"\nLabel '{label}' Scores:")
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']

    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)
# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Label '1' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Label '0' Scores:
Specificity: 1.0
Sensitivity: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0

Overall accuracy: 1.0


In [ ]:
# Initialize SVM classifier with a linear kernel
svm = SVC(kernel='linear')

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    svm.fit(X_train, y_train)

    # Predict on the test data
    y_pred = svm.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '1' Scores:
Specificity: 1.0
Sensitivity: 0.9958419958419958
Precision: 1.0
Recall: 0.9958419958419958
F1 Score: 0.9979166666666666
Accuracy: 0.9964221824686941

Label '0' Scores:
Specificity: 0.9958419958419958
Sensitivity: 1.0
Precision: 0.975
Recall: 1.0
F1 Score: 0.9873417721518987
Accuracy: 0.9964221824686941
Fold 2:

Label '1' Scores:
Specificity: 0.9834254143646409
Sensitivity: 0.9957264957264957
Precision: 0.9967914438502674
Recall: 0.9957264957264957
F1 Score: 0.9962586851950829
Accuracy: 0.9937332139659804

Label '0' Scores:
Specificity: 0.9957264957264957
Sensitivity: 0.9834254143646409
Precision: 0.978021978021978
Recall: 0.9834254143646409
F1 Score: 0.9807162534435261
Accuracy: 0.9937332139659804
Fold 3:

Label '1' Scores:
Specificity: 0.9942196531791907
Sensitivity: 1.0
Precision: 0.9989417989417989
Recall: 1.0
F1 Score: 0.9994706193753309
Accuracy: 0.999104744852283

Label '0' Scores:
Specificity: 1.0
Sensitivity: 0.9942196531791907
Precision: 1.0
Recall: 

#SVM Non-linear

In [ ]:
# Initialize SVM classifier with a non-linear kernel (e.g., 'rbf')
svm = SVC(kernel='rbf')

# Fit the classifier on the entire dataset
svm.fit(X, y)

# Predict on the same dataset
y_pred = svm.predict(X)

# Generate classification report
report = classification_report(y, y_pred, output_dict=True)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': 0, 'sensitivity': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'accuracy': 0} for label in labels}

# Generate confusion matrix
cm = confusion_matrix(y, y_pred, labels=labels)

# Extract evaluation metrics for each label
for idx, label in enumerate(labels):
    tp = cm[idx, idx]  # True Positive
    fn = cm[idx, :].sum() - tp  # False Negative
    fp = cm[:, idx].sum() - tp  # False Positive
    tn = cm.sum() - (tp + fn + fp)  # True Negative

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = report[str(label)]['precision']
    recall = report[str(label)]['recall']
    f1_score = report[str(label)]['f1-score']
    accuracy = report['accuracy']


# Print evaluation metrics for each label
print("Evaluation Metrics for SVM Non-Linear Classification:")
for label in labels:
    print(f"\nLabel '{label}' Scores:")
    print("Specificity:", specificity)
    print("Sensitivity:", sensitivity)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)
    print("Accuracy:", accuracy)

# Summarize results
print("\nOverall accuracy:", report['accuracy'])


Evaluation Metrics for SVM Non-Linear Classification:

Label '1' Scores:
Specificity: 0.9980984576378619
Sensitivity: 0.9859320046893317
Precision: 0.9894117647058823
Recall: 0.9859320046893317
F1 Score: 0.9876688197298884
Accuracy: 0.9962406015037594

Label '0' Scores:
Specificity: 0.9980984576378619
Sensitivity: 0.9859320046893317
Precision: 0.9894117647058823
Recall: 0.9859320046893317
F1 Score: 0.9876688197298884
Accuracy: 0.9962406015037594

Overall accuracy: 0.9962406015037594


In [ ]:
# Initialize SVM classifier with a non-linear kernel (RBF)
svm = SVC(kernel='rbf', probability=True)

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Get unique labels from the 'Outcome' column
labels = y.unique()

# Dictionary to store evaluation metrics
avg_scores = {label: {'specificity': [], 'sensitivity': [], 'precision': [], 'recall': [], 'f1_score': [], 'accuracy': []} for label in labels}

# Perform KFold cross-validation
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the classifier on the training data
    svm.fit(X_train, y_train)

    # Predict on the test data
    y_pred = svm.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    # Extract evaluation metrics for each label
    for idx, label in enumerate(labels):
        tp = cm[idx, idx]  # True Positive
        fn = cm[idx, :].sum() - tp  # False Negative
        fp = cm[:, idx].sum() - tp  # False Positive
        tn = cm.sum() - (tp + fn + fp)  # True Negative

        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision = report[str(label)]['precision']
        recall = report[str(label)]['recall']
        f1_score = report[str(label)]['f1-score']
        accuracy = report['accuracy']

        avg_scores[label]['specificity'].append(specificity)
        avg_scores[label]['sensitivity'].append(sensitivity)
        avg_scores[label]['precision'].append(precision)
        avg_scores[label]['recall'].append(recall)
        avg_scores[label]['f1_score'].append(f1_score)
        avg_scores[label]['accuracy'].append(accuracy)

    # Print evaluation metrics for each label after each iteration
    print(f"Fold {i+1}:")
    for label in labels:
        print(f"\nLabel '{label}' Scores:")
        print("Specificity:", avg_scores[label]['specificity'][-1])
        print("Sensitivity:", avg_scores[label]['sensitivity'][-1])
        print("Precision:", avg_scores[label]['precision'][-1])
        print("Recall:", avg_scores[label]['recall'][-1])
        print("F1 Score:", avg_scores[label]['f1_score'][-1])
        print("Accuracy:", avg_scores[label]['accuracy'][-1])

# Summarize results across all folds
print("\nAverage scores across all folds:")
for label in labels:
    print(f"\nLabel '{label}' Average Scores:")
    print("Specificity:", sum(avg_scores[label]['specificity']) / len(avg_scores[label]['specificity']))
    print("Sensitivity:", sum(avg_scores[label]['sensitivity']) / len(avg_scores[label]['sensitivity']))
    print("Precision:", sum(avg_scores[label]['precision']) / len(avg_scores[label]['precision']))
    print("Recall:", sum(avg_scores[label]['recall']) / len(avg_scores[label]['recall']))
    print("F1 Score:", sum(avg_scores[label]['f1_score']) / len(avg_scores[label]['f1_score']))
    print("Accuracy:", sum(avg_scores[label]['accuracy']) / len(avg_scores[label]['accuracy']))


Fold 1:

Label '1' Scores:
Specificity: 0.9871794871794872
Sensitivity: 0.997920997920998
Precision: 0.997920997920998
Recall: 0.997920997920998
F1 Score: 0.997920997920998
Accuracy: 0.9964221824686941

Label '0' Scores:
Specificity: 0.997920997920998
Sensitivity: 0.9871794871794872
Precision: 0.9871794871794872
Recall: 0.9871794871794872
F1 Score: 0.9871794871794872
Accuracy: 0.9964221824686941
Fold 2:

Label '1' Scores:
Specificity: 0.9834254143646409
Sensitivity: 0.9978632478632479
Precision: 0.9967982924226254
Recall: 0.9978632478632479
F1 Score: 0.997330485851575
Accuracy: 0.9955237242614146

Label '0' Scores:
Specificity: 0.9978632478632479
Sensitivity: 0.9834254143646409
Precision: 0.9888888888888889
Recall: 0.9834254143646409
F1 Score: 0.9861495844875346
Accuracy: 0.9955237242614146
Fold 3:

Label '1' Scores:
Specificity: 0.9826589595375722
Sensitivity: 0.9989406779661016
Precision: 0.9968287526427061
Recall: 0.9989406779661016
F1 Score: 0.9978835978835978
Accuracy: 0.996418979